In [10]:
from pulp import (
    LpProblem,
    LpMinimize,
    LpVariable,
    LpBinary,
    lpSum,
    PULP_CBC_CMD,
    LpStatus,
    LpContinuous,
)

In [11]:
# ---------------------
# Parámetros de ejemplo
# ---------------------
days = list(range(7))  # 0..6
shifts = list(range(4))  # 0:mañana, 1:mediodía, 2:tarde, 3:noche
workers = list(range(4))  # 0..3

SHIFT_NAMES = {0: "Mañana", 1: "Mediodía", 2: "Tarde", 3: "Noche"}

DAY_NAMES = {
    0: "Lunes",
    1: "Martes",
    2: "Miércoles",
    3: "Jueves",
    4: "Viernes",
    5: "Sábado",
    6: "Domingo",
}

In [19]:
# m[d][t]: mínimo de camareros requeridos
# Ejemplo: más carga el fin de semana y por la tarde/noche
m = {
    d: {
        0: 1 if d < 5 else 2,  # mañana
        1: 2 if d < 5 else 3,  # mediodía
        2: 1 if d < 5 else 2,  # tarde
        3: 2 if d < 5 else 3,  # noche
    }
    for d in days
}

# Disponibilidad a[w][d][t] (ejemplo: todos disponibles salvo alguna restricción artificial)
a = {w: {d: {t: 1 for t in shifts} for d in days} for w in workers}

# Ejemplo de indisponibilidad:
# Trabajador 0 no puede los domingos
for t in shifts:
    a[0][6][t] = 0

# Trabajador 1 no puede noches
for d in days:
    a[1][d][3] = 0

# Horas por turno
h = {
    0: 4,  # mañana de 8 a 13
    1: 3,  # mediodía de 13 a 16
    2: 5,  # tarde de 16 a 21
    3: 3,  # noche de 21 a 24
}

# Máximo de horas semanales por trabajador
Hmax = {w: 40 for w in workers}

In [20]:
# Pesos (ajustables)
P_COVER = 100.0  # penalizar déficit de cobertura
P_FULL = 10.0  # penalizar días completos
P_SPLIT = 5.0  # penalizar turnos partidos

# Big-M (suficientemente grande)
M_BIG = 4  # máximo nº de turnos por día es 4

In [21]:
# ---------------------
# Modelo
# ---------------------
model = LpProblem("Horario_Camareros", LpMinimize)

In [22]:
# Variables de asignación
x = {
    (w, d, t): LpVariable(f"x_{w}_{d}_{t}", cat=LpBinary)
    for w in workers
    for d in days
    for t in shifts
}

# Variables de déficit de cobertura
deficit = {
    (d, t): LpVariable(f"deficit_{d}_{t}", lowBound=0, cat=LpContinuous)
    for d in days
    for t in shifts
}

# Variables de violación de "día completo"
y_full = {
    (w, d): LpVariable(f"y_full_{w}_{d}", cat=LpBinary) for w in workers for d in days
}

# Variables de violación de "turnos partidos"
y_split_MT = {  # mañana-tarde sin mediodía
    (w, d): LpVariable(f"y_split_MT_{w}_{d}", cat=LpBinary)
    for w in workers
    for d in days
}

y_split_MN = {  # mediodía-noche sin tarde
    (w, d): LpVariable(f"y_split_MN_{w}_{d}", cat=LpBinary)
    for w in workers
    for d in days
}

In [23]:
# ---------------------
# Restricciones
# ---------------------

# 1) Cobertura mínima por turno (blanda con déficit)
for d in days:
    for t in shifts:
        model += (
            lpSum(x[(w, d, t)] for w in workers) + deficit[(d, t)] >= m[d][t],
            f"Cobertura_soft_d{d}_t{t}",
        )

# 2) Disponibilidad (dura)
for w in workers:
    for d in days:
        for t in shifts:
            model += (x[(w, d, t)] <= a[w][d][t], f"Disponibilidad_w{w}_d{d}_t{t}")

# 3) Máximo de horas semanales (dura)
for w in workers:
    model += (
        lpSum(h[t] * x[(w, d, t)] for d in days for t in shifts) <= Hmax[w],
        f"Max_horas_w{w}",
    )

# 4) No trabajar un día completo (blanda con y_full)
# Si y_full = 0 -> como antes: sum_t x <= 3
# Si y_full = 1 -> se permite llegar a 4
for w in workers:
    for d in days:
        model += (
            lpSum(x[(w, d, t)] for t in shifts) <= 3 + y_full[(w, d)],
            f"No_dia_completo_soft_w{w}_d{d}",
        )

# 5) No turnos partidos mañana-tarde sin mediodía (blanda)
# Original: x_M + x_T - x_MD <= 1
# Blando:   x_M + x_T - x_MD <= 1 + M_BIG * y_split_MT
for w in workers:
    for d in days:
        model += (
            x[(w, d, 0)] + x[(w, d, 2)] - x[(w, d, 1)]
            <= 1 + M_BIG * y_split_MT[(w, d)],
            f"No_partido_MT_soft_w{w}_d{d}",
        )

# 6) No turnos partidos mediodía-noche sin tarde (blanda)
# Original: x_MD + x_N - x_T <= 1
# Blando:   x_MD + x_N - x_T <= 1 + M_BIG * y_split_MN
for w in workers:
    for d in days:
        model += (
            x[(w, d, 1)] + x[(w, d, 3)] - x[(w, d, 2)]
            <= 1 + M_BIG * y_split_MN[(w, d)],
            f"No_partido_MN_soft_w{w}_d{d}",
        )

# 7) Noche y mañana siguiente incompatibles (dura)
for w in workers:
    for d in range(6):  # hasta el sábado
        model += (x[(w, d, 3)] + x[(w, d + 1, 0)] <= 1, f"Noche_maniana_sig_w{w}_d{d}")


In [24]:
# ---------------------
# Función objetivo
# ---------------------
# Minimizar:
#  - penalización por déficit de cobertura
#  - penalización por días completos
#  - penalización por turnos partidos

obj = (
    lpSum(P_COVER * deficit[(d, t)] for d in days for t in shifts)
    + lpSum(P_FULL * y_full[(w, d)] for w in workers for d in days)
    + lpSum(
        P_SPLIT * (y_split_MT[(w, d)] + y_split_MN[(w, d)])
        for w in workers
        for d in days
    )
)

model += obj, "Min_penalizaciones"

In [ ]:
# ---------------------
# Resolver
# ---------------------
model.solve(PULP_CBC_CMD(msg=False))

print("Estado de la solución:", LpStatus[model.status])

if LpStatus[model.status] == "Optimal":
    # Imprimir horarios
    for d in days:
        print(f"\n=== {DAY_NAMES[d]} ===")
        for t in shifts:
            asignados = [w for w in workers if x[(w, d, t)].value() == 1]
            print(
                f"  {SHIFT_NAMES[t]}: Trabajadores {asignados if asignados else '(ninguno)'}"
            )

    # Mostrar algunas métricas de violación
    print("\n--- Déficits de cobertura ---")
    for d in days:
        for t in shifts:
            val = deficit[(d, t)].value()
            if val > 1e-3:
                print(
                    f"Déficit en {DAY_NAMES[d]} {SHIFT_NAMES[t]}: {val:.2f} camareros"
                )

    print("\n--- Días completos usados ---")
    for w in workers:
        for d in days:
            if y_full[(w, d)].value() > 0.5:
                print(f"Trabajador {w} trabaja día completo el {DAY_NAMES[d]}")

    print("\n--- Turnos partidos permitidos ---")
    for w in workers:
        for d in days:
            if y_split_MT[(w, d)].value() > 0.5:
                print(
                    f"Trabajador {w} tiene partido Mañana-Tarde sin Mediodía el {DAY_NAMES[d]}"
                )
            if y_split_MN[(w, d)].value() > 0.5:
                print(
                    f"Trabajador {w} tiene partido Mediodía-Noche sin Tarde el {DAY_NAMES[d]}"
                )
else:
    print("No se encontró solución óptima/viable.")

Estado de la solución: Optimal

=== Lunes ===
  Mañana: Trabajadores [1]
  Mediodía: Trabajadores [0, 2]
  Tarde: Trabajadores [0]
  Noche: Trabajadores [2, 3]

=== Martes ===
  Mañana: Trabajadores [0]
  Mediodía: Trabajadores [1, 2]
  Tarde: Trabajadores (ninguno)
  Noche: Trabajadores [0, 3]

=== Miércoles ===
  Mañana: Trabajadores [1]
  Mediodía: Trabajadores [0, 1]
  Tarde: Trabajadores [1]
  Noche: Trabajadores [2, 3]

=== Jueves ===
  Mañana: Trabajadores [0]
  Mediodía: Trabajadores [1, 2]
  Tarde: Trabajadores (ninguno)
  Noche: Trabajadores [0, 3]

=== Viernes ===
  Mañana: Trabajadores [1]
  Mediodía: Trabajadores [0, 3]
  Tarde: Trabajadores [0]
  Noche: Trabajadores [2, 3]

=== Sábado ===
  Mañana: Trabajadores [0, 1]
  Mediodía: Trabajadores [1, 2, 3]
  Tarde: Trabajadores [2, 3]
  Noche: Trabajadores [0, 2, 3]

=== Domingo ===
  Mañana: Trabajadores [1]
  Mediodía: Trabajadores [1, 2, 3]
  Tarde: Trabajadores [2, 3]
  Noche: Trabajadores [2, 3]

--- Déficits de cobertur